In [1]:
!pip install trl unsloth comet-ml>=3.43.2

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
torchaudio 2.6.0+cu124 requires torch==2.6.0, but you have torch 2.7.1 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
fastai 2.7.19 requires torch<2.7,>=1.10, but you have torch 2.7.1 which is incompatible.
bigframes 2.8.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.31.0, but you have google-cloud-bigquery 3.25.0 which is incompatible.
bigframes 2.8.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.


In [2]:
from unsloth import PatchDPOTrainer

PatchDPOTrainer()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-07-22 01:03:45.227151: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753146225.430712      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753146225.493254      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
comet_ml is installed but the Comet API Key is not configured. Please set the `COMET_API_KEY` environment variable to enable Comet logging. Check out the documentation for other ways of configuring it: https://www.comet.com/docs/v2/guides/experiment-management/configure-sdk/#set-the-api-key


🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
import os
import torch
from datasets import load_dataset
from transformers import TrainingArguments, TextStreamer
from unsloth import FastLanguageModel, is_bfloat16_supported
from trl import DPOConfig, DPOTrainer

In [4]:
max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="SkillRipper/TwinLlama-3.2-3B-Instruct",
    max_seq_length=max_seq_length,
    load_in_4bit=False,
)

==((====))==  Unsloth 2025.7.6: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 6.0. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
)

Unsloth 2025.7.6 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [6]:
 dataset = load_dataset("SkillRipper/preference-data", split="train")

README.md:   0%|          | 0.00/369 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/242k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1110 [00:00<?, ? examples/s]

In [7]:
alpaca_template = """
Below is an instruction that describes a task. 
Write a response that appropriately completes the request.
 ### Instruction:
 {}
 ### Response:
"""

EOS_TOKEN = tokenizer.eos_token

def format_samples(example):
    example["prompt"] = alpaca_template.format(example["prompt"])
    example["chosen"] = example['chosen'] + EOS_TOKEN
    example["rejected"] = example['rejected'] + EOS_TOKEN
    
    return {"prompt": example["prompt"], "chosen": 

example["chosen"], "rejected": example["rejected"]}
dataset = dataset.map(format_samples)
dataset = dataset.train_test_split(test_size=0.05)

Map:   0%|          | 0/1110 [00:00<?, ? examples/s]

In [8]:
trainer = DPOTrainer(
    model=model,
    ref_model=None,
    tokenizer=tokenizer,
    beta=0.5,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    max_length=max_seq_length//2,
    max_prompt_length=max_seq_length//2,
    args=DPOConfig(
        learning_rate=2e-6,
        lr_scheduler_type="linear",
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps=8,
        num_train_epochs=1,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        optim="adamw_8bit",
        weight_decay=0.01,
        warmup_steps=10,
        output_dir="output",
        eval_strategy="steps",
        eval_steps=0.2,
        logging_steps=1,
        report_to="none",
        seed=0,
    ),
 )
trainer.train()

Extracting prompt in train dataset (num_proc=2):   0%|          | 0/1054 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/1054 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/1054 [00:00<?, ? examples/s]

Extracting prompt in eval dataset (num_proc=2):   0%|          | 0/56 [00:00<?, ? examples/s]

Applying chat template to eval dataset (num_proc=2):   0%|          | 0/56 [00:00<?, ? examples/s]

...

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,054 | Num Epochs = 1 | Total steps = 66
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)


Step,Training Loss,Validation Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss,aux_loss
14,0.692200,0.690605,-0.001360,-0.006460,0.821429,0.005100,-138.949600,-82.829979,-0.321397,-0.091277,0,0,0,0
28,0.686600,0.682729,-0.006674,-0.027737,0.803571,0.021062,-139.002716,-83.042732,-0.323317,-0.093832,No Log,No Log,No Log,No Log
42,0.677400,0.674293,-0.013561,-0.052001,0.839286,0.038440,-139.071594,-83.285385,-0.325163,-0.095991,No Log,No Log,No Log,No Log
56,0.673100,0.669017,-0.019273,-0.068679,0.875000,0.049406,-139.128708,-83.452164,-0.326521,-0.097750,No Log,No Log,No Log,No Log


Unsloth: Will smartly offload gradients to save VRAM!


comet_ml is installed but the Comet API Key is not configured. Please set the `COMET_API_KEY` environment variable to enable Comet logging. Check out the documentation for other ways of configuring it: https://www.comet.com/docs/v2/guides/experiment-management/configure-sdk/#set-the-api-key


TrainOutput(global_step=66, training_loss=0.6810455114552469, metrics={'train_runtime': 960.0786, 'train_samples_per_second': 1.098, 'train_steps_per_second': 0.069, 'total_flos': 0.0, 'train_loss': 0.6810455114552469, 'epoch': 1.0})

In [9]:
FastLanguageModel.for_inference(model)
message = alpaca_template.format("Write a paragraph to introduce supervised fine-tuning.", "")
inputs = tokenizer([message], return_tensors="pt").to("cuda")
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=256, use_cache=True)

<|begin_of_text|>
Below is an instruction that describes a task. 
Write a response that appropriately completes the request.
 ### Instruction:
 Write a paragraph to introduce supervised fine-tuning.
 ### Response:
 Supervised fine-tuning is a method used to adapt a pre-trained model to a specific task. In this process, the pre-trained model is trained on a labeled dataset to learn the task-specific patterns and relationships. The model learns to recognize and classify the task-specific features, and the trained model is then used for prediction or generation tasks. Supervised fine-tuning is a widely used technique in deep learning, especially for natural language processing tasks.<|eot_id|>


In [10]:
model.save_pretrained_merged("model", tokenizer, save_method="merged_16bit")

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Successfully copied all 2 files from cache to model.


Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [00:38<00:00, 19.29s/it]


In [ ]:
from huggingface_hub import login
login(token="")

In [12]:
model.push_to_hub_merged("SkillRipper/TwinLlama-3.2-3B-DPO", tokenizer, save_method="merged_16bit")

  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Successfully copied all 2 files from cache to SkillRipper/TwinLlama-3.2-3B-DPO.


Unsloth: Merging weights into 16bit:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit:  50%|█████     | 1/2 [01:08<01:08, 68.83s/it]

  0%|          | 0/1 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [01:29<00:00, 44.95s/it]
